## Generación de archivos específicos para análisis

En este apartado generamos archivos CSV específicos para los distintos tipos de análisis requeridos.

In [ ]:
# Este código generaría y guardaría los archivos de análisis específicos
# En una implementación real, esto se ejecutaría después de procesar todo el historial

if 'results_df' in locals() and len(results_df) > 0:
    # 1. Archivo de promedio de calidad por tipo de medio
    promedio_calidad_tipo = results_df.groupby('Tipo_Medio_TMDb')['Calificacion_Promedio_TMDb'].mean().reset_index()
    print("\n--- Promedio de Calificación por Tipo de Medio ---")
    display(promedio_calidad_tipo)
    
    # Guardar en archivo
    try:
        promedio_calidad_tipo.to_csv('../data/processed/promedio_calidad_tipo.csv', index=False)
        print("✅ Archivo 'promedio_calidad_tipo.csv' guardado exitosamente.")
    except Exception as e:
        print(f"❌ Error al guardar 'promedio_calidad_tipo.csv': {e}")
        
    # 2. Archivo de resumen con conteo por categoría de calidad y tipo
    resumen_calidad_tipo = results_df.groupby(['Tipo_Medio_TMDb', 'Calidad']).size().reset_index(name='Conteo')
    print("\n--- Resumen de Conteo por Calidad y Tipo de Medio ---")
    display(resumen_calidad_tipo)
    
    # Guardar en archivo
    try:
        resumen_calidad_tipo.to_csv('../data/processed/resumen_calidad_tipo.csv', index=False)
        print("✅ Archivo 'resumen_calidad_tipo.csv' guardado exitosamente.")
    except Exception as e:
        print(f"❌ Error al guardar 'resumen_calidad_tipo.csv': {e}")
        
    # 3. Archivo extendido para análisis de géneros (una fila por género)
    if 'df_generos' in locals() and len(df_generos) > 0:
        # Opcional: agregar análisis de géneros más populares
        generos_populares = df_generos.groupby('Genero').agg({
            'Popularidad_TMDb': 'mean',
            'Calificacion_Promedio_TMDb': 'mean',
            'Genero': 'count'
        }).rename(columns={'Genero': 'Conteo'}).sort_values('Conteo', ascending=False).reset_index()
        
        print("\n--- Géneros Más Populares ---")
        display(generos_populares.head(10))
        
        # Guardar en archivo
        try:
            df_generos.to_csv('../data/processed/netflix_analisis_generos.csv', index=False)
            print("✅ Archivo 'netflix_analisis_generos.csv' guardado exitosamente.")
            
            generos_populares.to_csv('../data/processed/generos_populares.csv', index=False)
            print("✅ Archivo 'generos_populares.csv' guardado exitosamente.")
        except Exception as e:
            print(f"❌ Error al guardar archivos de géneros: {e}")
    else:
        print("⚠️ No hay datos de géneros para guardar.")
        
    # 4. Guardar el DataFrame principal procesado
    try:
        results_df.to_csv('../data/processed/netflix_eda_processed.csv', index=False)
        print("✅ Archivo principal 'netflix_eda_processed.csv' guardado exitosamente.")
    except Exception as e:
        print(f"❌ Error al guardar el archivo principal: {e}")
else:
    print("⚠️ No hay datos para generar los archivos de análisis.")


In [ ]:
# --- PASO 1: Cargar el historial de Netflix ---
# Esto leería el archivo NetflixViewingHistory.csv real
viewing_history_path = '../data/raw/NetflixViewingHistory.csv'
try:
    netflix_history = pd.read_csv(viewing_history_path)
    print(f"Cargados {len(netflix_history)} registros del historial de Netflix.")
    display(netflix_history.head())
except Exception as e:
    print(f"Error al cargar el historial: {e}")
    # Crear un DataFrame de ejemplo si no podemos cargar el archivo
    netflix_history = pd.DataFrame({
        "Title": example_titles,
        "Date": ["5/5/25", "4/4/25", "3/3/25", "2/2/25"]
    })
    print("Usando datos de ejemplo.")

# --- PASO 2: Procesar títulos para mejor búsqueda ---
def limpiar_titulo(titulo):
    """Limpia y simplifica un título para búsqueda en API."""
    # Quitar temporadas, episodios, etc.
    import re
    # Patrones para remover: "Temporada X", "Episodio X", "Parte X", etc.
    patrones = [
        r': Parte \d+.*$',    # Quita ": Parte X" y todo lo que sigue
        r': T\d+.*$',         # Quita ": T1" y todo lo que sigue (temporadas)
        r': Temporada \d+.*$', # Quita ": Temporada X" y todo lo que sigue
        r': E\d+.*$',         # Quita ": E1" y todo lo que sigue (episodios)
        r': Episodio \d+.*$',  # Quita ": Episodio X" y todo lo que sigue
        r': Cap[ií]tulo \d+.*$', # Quita ": Capítulo X" y todo lo que sigue
    ]
    
    titulo_limpio = titulo
    for patron in patrones:
        titulo_limpio = re.sub(patron, '', titulo_limpio, flags=re.IGNORECASE)
    
    return titulo_limpio.strip()

# Aplicar limpieza a los títulos
netflix_history['Titulo_Limpio_Busqueda'] = netflix_history['Title'].apply(limpiar_titulo)
print("\nTítulos procesados para búsqueda:")
display(pd.DataFrame({
    'Título Original': netflix_history['Title'],
    'Título para Búsqueda': netflix_history['Titulo_Limpio_Busqueda']
}).head())

# --- PASO 3: Enriquecer con la API TMDb (proceso principal) ---
# En una implementación real, recorreríamos los títulos del historial
# El código sería similar al ejemplo de prueba pero procesando netflix_history

# --- PASO 4: Procesamiento final para el dashboard ---
# Aquí mostramos cómo se vería el procesamiento final

# Simulación de resultado enriquecido
if 'results_df' in locals():
    # Asegurarse de que todas las fechas estén en formato adecuado
    results_df['Fecha_Visualizacion'] = pd.to_datetime(results_df['Fecha_Visualizacion'])
    results_df['Fecha_Estreno_TMDb'] = pd.to_datetime(results_df['Fecha_Estreno_TMDb'], errors='coerce')
    
    # Convertir valores numéricos
    numeric_cols = ['Popularidad_TMDb', 'Calificacion_Promedio_TMDb', 
                    'Cantidad_Votos_TMDb', 'Duracion_Minutos_TMDb']
    for col in numeric_cols:
        results_df[col] = pd.to_numeric(results_df[col], errors='coerce')
    
    # ----- NUEVAS COLUMNAS PARA ANÁLISIS -----
    
    # Extraer componentes de fecha para análisis temporal
    results_df['Anio'] = results_df['Fecha_Visualizacion'].dt.year
    results_df['Mes_Num'] = results_df['Fecha_Visualizacion'].dt.month
    results_df['Dia_Mes'] = results_df['Fecha_Visualizacion'].dt.day
    results_df['Dia_Semana_Num'] = results_df['Fecha_Visualizacion'].dt.dayofweek  # 0=Lunes, 6=Domingo
    results_df['Hora_Visualizacion'] = results_df['Fecha_Visualizacion'].dt.hour
    results_df['Semana_Anio'] = results_df['Fecha_Visualizacion'].dt.isocalendar().week
    
    # Nombres en español para meses y días
    meses_es = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
                7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
    dias_es = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
    
    results_df['Mes'] = results_df['Mes_Num'].map(meses_es)
    results_df['Dia_Semana'] = results_df['Dia_Semana_Num'].map(dias_es)
    
    # Añadir clasificación de calidad (según calificación) con 4 niveles
    condiciones_detalladas = [
        (results_df['Calificacion_Promedio_TMDb'] >= 8.0),
        (results_df['Calificacion_Promedio_TMDb'] >= 7.0) & (results_df['Calificacion_Promedio_TMDb'] < 8.0),
        (results_df['Calificacion_Promedio_TMDb'] >= 6.0) & (results_df['Calificacion_Promedio_TMDb'] < 7.0),
        (results_df['Calificacion_Promedio_TMDb'] < 6.0)
    ]
    categorias_detalladas = ['Excelente', 'Bueno', 'Regular', 'Malo']
    results_df['Categoria_Calidad'] = np.select(condiciones_detalladas, categorias_detalladas, default='Sin calificación')
    
    # Añadir clasificación simplificada de calidad (Alta, Media, Baja)
    condiciones_simples = [
        (results_df['Calificacion_Promedio_TMDb'] >= 7.5),
        (results_df['Calificacion_Promedio_TMDb'] >= 6.0) & (results_df['Calificacion_Promedio_TMDb'] < 7.5),
        (results_df['Calificacion_Promedio_TMDb'] < 6.0)
    ]
    categorias_simples = ['Alta', 'Media', 'Baja']
    results_df['Calidad'] = np.select(condiciones_simples, categorias_simples, default='Sin calificación')
    
    # Simplificar tipo de medio (solo movie o tv)
    results_df['Tipo_Medio'] = results_df['Tipo_Medio_TMDb']
    
    # Indicadores para análisis de tipo de contenido
    results_df['Es_Serie'] = results_df['Tipo_Medio_TMDb'] == 'tv'
    results_df['Es_Pelicula'] = results_df['Tipo_Medio_TMDb'] == 'movie'
    
    # Tiempo desde el estreno hasta la visualización (en días)
    results_df['Tiempo_Desde_Estreno'] = (results_df['Fecha_Visualizacion'] - results_df['Fecha_Estreno_TMDb']).dt.days
    
    # Tiempo de visualización estimado (en minutos) - Simulación
    # En datos reales, esto vendría de la API de Netflix o de otra fuente
    # Para películas: usamos la duración completa
    # Para series: usamos un porcentaje aleatorio de la duración para simular
    import random
    
    def estimar_tiempo_visto(row):
        if pd.isna(row['Duracion_Minutos_TMDb']) or row['Duracion_Minutos_TMDb'] == 0:
            return None
            
        if row['Tipo_Medio_TMDb'] == 'movie':
            # Para películas, asumimos entre 70-100% de visualización
            return round(row['Duracion_Minutos_TMDb'] * random.uniform(0.7, 1.0))
        else:
            # Para series, asumimos entre 80-100% de visualización de un episodio
            return round(row['Duracion_Minutos_TMDb'] * random.uniform(0.8, 1.0))
    
    # Aplicar la función para estimar tiempo visto
    results_df['Tiempo_Visualizacion'] = results_df.apply(estimar_tiempo_visto, axis=1)
    
    print("\n--- DataFrame Final Procesado para el Dashboard ---")
    display(results_df.head())
    
    print("\nColumnas en el DataFrame procesado:")
    print(results_df.columns.tolist())
    
    # --- GENERAR DATAFRAMES ADICIONALES PARA ANÁLISIS ESPECÍFICOS ---
    
    # 1. DataFrame para análisis de calidad
    df_calidad = results_df[['Tipo_Medio', 'Calidad', 'Calificacion_Promedio_TMDb', 
                            'Popularidad_TMDb', 'Fecha_Visualizacion', 'Anio']].copy()
    
    print("\n--- DataFrame para Análisis de Calidad ---")
    display(df_calidad.head())
    
    # 2. DataFrame para análisis temporal
    df_tiempo = results_df[['Fecha_Visualizacion', 'Anio', 'Mes', 'Dia_Semana',
                           'Duracion_Minutos_TMDb', 'Popularidad_TMDb', 'Tiempo_Visualizacion']].copy()
    
    print("\n--- DataFrame para Análisis Temporal ---")
    display(df_tiempo.head())
    
    # 3. DataFrame para análisis de géneros (una fila por género)
    # Primero, explotar la columna de géneros
    if 'Generos_TMDb' in results_df.columns:
        # Crear una lista de DataFrames, uno por cada fila con sus géneros explotados
        generos_dfs = []
        
        for idx, row in results_df.iterrows():
            generos = [g.strip() for g in row['Generos_TMDb'].split(',')] if isinstance(row['Generos_TMDb'], str) and row['Generos_TMDb'] else ['Sin género']
            
            for genero in generos:
                if genero:  # Solo considerar géneros no vacíos
                    genero_row = {
                        'Genero': genero,
                        'Popularidad_TMDb': row['Popularidad_TMDb'],
                        'Calificacion_Promedio_TMDb': row['Calificacion_Promedio_TMDb'],
                        'Tipo_Medio': row['Tipo_Medio'],
                        'Fecha_Visualizacion': row['Fecha_Visualizacion'],
                        'Anio': row['Anio'],
                        'Titulo_TMDb': row['Titulo_TMDb']
                    }
                    generos_dfs.append(pd.DataFrame([genero_row]))
        
        # Concatenar todos los DataFrames
        if generos_dfs:
            df_generos = pd.concat(generos_dfs, ignore_index=True)
            
            print("\n--- DataFrame para Análisis de Géneros (una fila por género) ---")
            display(df_generos.head())
            
            # Opcional: Guardar cada DataFrame en un archivo CSV separado
            # df_calidad.to_csv('../data/processed/netflix_analisis_calidad.csv', index=False)
            # df_tiempo.to_csv('../data/processed/netflix_analisis_tiempo.csv', index=False)
            # df_generos.to_csv('../data/processed/netflix_analisis_generos.csv', index=False)
            
            # Generar análisis de promedio de calidad por tipo
            promedio_calidad_tipo = results_df.groupby('Tipo_Medio_TMDb')['Calificacion_Promedio_TMDb'].mean().reset_index()
            print("\n--- Promedio de Calificación por Tipo de Medio ---")
            display(promedio_calidad_tipo)
            # Opcional: Guardar este análisis
            # promedio_calidad_tipo.to_csv('../data/processed/promedio_calidad_tipo.csv', index=False)
        else:
            print("No se pudo crear el DataFrame de géneros porque no hay datos de géneros válidos.")
    else:
        print("No se encontró la columna de géneros en los datos.")
else:
    print("No hay datos enriquecidos para procesar.")

## LIBRERIAS

Importación de las librerías necesarias para el enriquecimiento de datos.

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import time
import numpy as np

In [ ]:
# --- Carga de API Token (como antes) ---
load_dotenv()
api_token = os.getenv('TMDB_API_READ_ACCESS_TOKEN')
if not api_token:
    raise ValueError("Error: No se encontró TMDB_API_READ_ACCESS_TOKEN en el archivo .env")
print("API Token (v4) cargado exitosamente.")

# --- Lista de títulos de ejemplo para buscar ---
# (Más adelante, esto vendrá de tu archivo NetflixViewingHistory.csv)
example_titles = ["La casa de papel", "Forrest Gump", "Nuestro Planeta", "Título Inventado Que No Existe"]

# --- Lista para almacenar los resultados enriquecidos ---
enriched_data_list = []

# --- Headers para la API (constante) ---
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {api_token}"
}

# --- Iterar sobre los títulos de ejemplo ---
for title_to_search in example_titles:
    print(f"\n--- Procesando: {title_to_search} ---")
    
    # 1. BUSCAR con /search/multi
    search_url = "https://api.themoviedb.org/3/search/multi"
    search_params = {'query': title_to_search, 'language': 'es-ES'}
    best_match_id = None
    best_match_type = None
    
    try:
        response_search = requests.get(search_url, params=search_params, headers=headers)
        response_search.raise_for_status()
        search_results = response_search.json()

        # Encontrar el primer resultado relevante (movie/tv)
        for result in search_results.get('results', []):
            media_type = result.get('media_type')
            if media_type in ['movie', 'tv']:
                best_match_id = result.get('id')
                best_match_type = media_type
                print(f"Encontrado: ID={best_match_id}, Tipo={best_match_type}")
                break
        
        if not best_match_id:
            print(f"No se encontró coincidencia relevante para '{title_to_search}'.")
            continue # Pasar al siguiente título del bucle

    except requests.exceptions.RequestException as e:
        print(f"Error en búsqueda API para '{title_to_search}': {e}")
        continue # Pasar al siguiente título

    # Pequeña pausa entre búsqueda y detalles
    time.sleep(0.3) 

    # 2. OBTENER DETALLES si se encontró ID y Tipo
    if best_match_id and best_match_type:
        details_url = f"https://api.themoviedb.org/3/{best_match_type}/{best_match_id}"
        details_params = {'language': 'es-ES'}
        
        try:
            response_details = requests.get(details_url, params=details_params, headers=headers)
            response_details.raise_for_status()
            details = response_details.json()

            # 3. EXTRAER Y GUARDAR DATOS en un diccionario con nombres de columnas alineados con el dashboard
            data_entry = {} # Diccionario para este título
            data_entry['Titulo_Original_Netflix'] = title_to_search  # Título original de Netflix
            data_entry['Titulo_Limpio_Busqueda'] = title_to_search   # Título limpio usado para la búsqueda
            data_entry['ID_TMDb'] = best_match_id
            data_entry['Tipo_Medio_TMDb'] = best_match_type
            
            # Título y Fecha
            data_entry['Titulo_TMDb'] = details.get('title') if best_match_type == 'movie' else details.get('name', 'N/A')
            data_entry['Fecha_Estreno_TMDb'] = details.get('release_date') if best_match_type == 'movie' else details.get('first_air_date', 'N/A')
            
            # Géneros - Convertirlos a string separado por comas
            genres_list = details.get('genres', [])
            genres_names = [genre['name'] for genre in genres_list]
            data_entry['Generos_TMDb'] = ', '.join(genres_names) if genres_names else ''
            
            # Popularidad y Datos de votación
            data_entry['Popularidad_TMDb'] = details.get('popularity', 0.0)
            data_entry['Calificacion_Promedio_TMDb'] = details.get('vote_average', 0.0)
            data_entry['Cantidad_Votos_TMDb'] = details.get('vote_count', 0)
            
            # Duración (Runtime) - Lógica unificada
            runtime = 0 # Default
            if best_match_type == 'movie':
                runtime = details.get('runtime', 0)
            elif best_match_type == 'tv':
                episode_runtime = details.get('episode_run_time', [])
                if episode_runtime:
                    runtime = episode_runtime[0] # Tomamos el primer valor (más común)
            data_entry['Duracion_Minutos_TMDb'] = float(runtime) if runtime else 0.0 # Asegurar que sea numérico
            
            # Tipos simplificados y calificaciones adicionales
            data_entry['Tipo_Medio'] = best_match_type  # Mismo valor que Tipo_Medio_TMDb
            
            # Clasificaciones de calidad (se procesarán completamente más adelante)
            vote_avg = details.get('vote_average', 0.0)
            if vote_avg >= 7.5:
                data_entry['Calidad'] = 'Alta'
            elif vote_avg >= 6.0:
                data_entry['Calidad'] = 'Media'
            elif vote_avg > 0:
                data_entry['Calidad'] = 'Baja'
            else:
                data_entry['Calidad'] = 'Sin calificación'
            
            # Añadir fecha de visualización simulada (esto sería reemplazado con datos reales)
            # En una implementación real, esto vendría del archivo NetflixViewingHistory.csv
            import random
            from datetime import datetime, timedelta
            
            # Generar fechas aleatorias entre 2023 y 2025
            start_date = datetime(2023, 1, 1)
            end_date = datetime(2025, 5, 10)
            days_between = (end_date - start_date).days
            random_days = random.randint(0, days_between)
            random_hours = random.randint(0, 23)
            random_minutes = random.randint(0, 59)
            
            random_date = start_date + timedelta(days=random_days, hours=random_hours, minutes=random_minutes)
            data_entry['Fecha_Visualizacion'] = random_date.strftime('%Y-%m-%d %H:%M:%S')
            
            # Añadir el diccionario a nuestra lista
            enriched_data_list.append(data_entry)
            print(f"Datos para '{data_entry['Titulo_TMDb']}' añadidos.")

        except requests.exceptions.RequestException as e:
            print(f"Error obteniendo detalles API para ID {best_match_id}: {e}")
        except Exception as e:
            print(f"Error procesando detalles para ID {best_match_id}: {e}")

    # Pausa final antes de la siguiente iteración
    time.sleep(0.3)


# --- Convertir la lista de resultados a un DataFrame de Pandas ---
if enriched_data_list:
    results_df = pd.DataFrame(enriched_data_list)
    
    # Asegurar que las columnas estén en el orden correcto y coincidan con el formato esperado
    columnas_deseadas = [
        'Titulo_Original_Netflix',
        'Fecha_Visualizacion',
        'Titulo_Limpio_Busqueda',
        'ID_TMDb',
        'Titulo_TMDb',
        'Generos_TMDb',
        'Popularidad_TMDb',
        'Calificacion_Promedio_TMDb',
        'Cantidad_Votos_TMDb',
        'Tipo_Medio_TMDb',
        'Fecha_Estreno_TMDb',
        'Duracion_Minutos_TMDb'
    ]
    
    # Crear columnas faltantes si es necesario
    for col in columnas_deseadas:
        if col not in results_df.columns:
            results_df[col] = None
    
    # Reordenar columnas
    results_df = results_df[columnas_deseadas]
    
    print("\n--- DataFrame con los Datos Enriquecidos (Formato Final) ---")
    display(results_df)
    
    # En una implementación real, guardaríamos el DataFrame a un archivo CSV
    # results_df.to_csv('../data/processed/netflix_viewing_enriched.csv', index=False)
    print("\nEste DataFrame se puede guardar como 'netflix_viewing_enriched.csv'")
    print("Y luego procesarse para generar el archivo final 'netflix_eda_processed.csv'")
else:
    print("\nNo se pudo enriquecer ningún dato en esta prueba.")

API Token (v4) cargado exitosamente.

--- Procesando: La casa de papel ---
Encontrado: ID=71446, Tipo=tv
Datos para 'La casa de papel' añadidos.

--- Procesando: Forrest Gump ---
Encontrado: ID=13, Tipo=movie
Datos para 'Forrest Gump' añadidos.

--- Procesando: Nuestro Planeta ---
Encontrado: ID=75782, Tipo=tv
Datos para 'Nuestro planeta (One Strange Rock)' añadidos.

--- Procesando: Título Inventado Que No Existe ---
No se encontró coincidencia relevante para 'Título Inventado Que No Existe'.

--- DataFrame con los Datos Enriquecidos (Ejemplo) ---


,search_query,tmdb_id,media_type,fetched_title,release_date,genres,runtime_minutes,vote_average,poster_path
0,La casa de papel,71446,tv,La casa de papel,2017-05-02,"[Crimen, Drama]",0,8.237,/4lYoeY8Ukux2EIL18izTeoW8m6P.jpg
1,Forrest Gump,13,movie,Forrest Gump,1994-06-23,"[Comedia, Drama, Romance]",142,8.468,/oiqKEhEfxl9knzWXvWecJKN3aj6.jpg
2,Nuestro Planeta,75782,tv,Nuestro planeta (One Strange Rock),2018-03-26,[Documental],47,7.600,/5bJYVZyprV0iXgqvEDYzNSY7opt.jpg


## Ejemplo del proceso completo (para implementación real)

En una implementación real, estos serían los pasos completos para procesar los datos de Netflix: